# Datan esikäsittely

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./temp/used_cars_data.csv', usecols= ['vin', 'body_type', 'city_fuel_economy', 'highway_fuel_economy', 'engine_type', 'exterior_color', 'frame_damaged', 'has_accidents', 'salvage', 'fuel_tank_volume', 'fuel_type', 'horsepower', 'isCab', 'make_name', 'maximum_seating', 'mileage', 'model_name', 'major_options', 'seller_rating', 'price', 'torque', 'transmission', 'wheel_system', 'year'])
df.head()

,vin,body_type,city_fuel_economy,engine_type,exterior_color,frame_damaged,fuel_tank_volume,fuel_type,has_accidents,highway_fuel_economy,...,maximum_seating,mileage,model_name,price,salvage,seller_rating,torque,transmission,wheel_system,year
0,ZACNJABB5KPJ92081,SUV / Crossover,NaN,I4,Solar Yellow,NaN,12.7 gal,Gasoline,NaN,NaN,...,5 seats,7.0,Renegade,23141.0,NaN,2.8,"200 lb-ft @ 1,750 RPM",A,FWD,2019
1,SALCJ2FX1LH858117,SUV / Crossover,NaN,I4,Narvik Black,NaN,17.7 gal,Gasoline,NaN,NaN,...,7 seats,8.0,Discovery Sport,46500.0,NaN,3.0,"269 lb-ft @ 1,400 RPM",A,AWD,2020
2,JF1VA2M67G9829723,Sedan,17.0,H4,None,False,15.9 gal,Gasoline,False,23.0,...,5 seats,NaN,WRX STI,46995.0,False,NaN,"290 lb-ft @ 4,000 RPM",M,AWD,2016
3,SALRR2RV0L2433391,SUV / Crossover,NaN,V6,Eiger Gray,NaN,23.5 gal,Gasoline,NaN,NaN,...,7 seats,11.0,Discovery,67430.0,NaN,3.0,"332 lb-ft @ 3,500 RPM",A,AWD,2020
4,SALCJ2FXXLH862327,SUV / Crossover,NaN,I4,Narvik Black,NaN,17.7 gal,Gasoline,NaN,NaN,...,7 seats,7.0,Discovery Sport,48880.0,NaN,3.0,"269 lb-ft @ 1,400 RPM",A,AWD,2020


## Poistetaan duplikaattirivit

Poistetaan rivit, joissa vin on sama. Tämän jälkeen poistetaan myös vin datasetistä, koska emme käytä sitä.

In [2]:
duplicate_rows = df.duplicated().sum()
print(f"Duplicate rows count: {duplicate_rows}")

Duplicate rows count: 40


In [3]:
df = df.drop_duplicates().copy()
dup = df.duplicated().sum()
df.drop(['vin'], axis=1, inplace=True)
print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [4]:
print((df.isna().sum()/df.shape[0]*100).sort_values(ascending=False))

has_accidents           47.552633
frame_damaged           47.552633
isCab                   47.552633
salvage                 47.552633
torque                  17.259600
highway_fuel_economy    16.375933
city_fuel_economy       16.375933
major_options            6.668200
horsepower               5.746100
fuel_tank_volume         5.308867
maximum_seating          5.308867
wheel_system             4.891000
mileage                  4.812900
engine_type              3.352667
fuel_type                2.757467
transmission             2.139467
seller_rating            1.362400
body_type                0.451400
exterior_color           0.000867
price                    0.000000
model_name               0.000000
make_name                0.000000
year                     0.000000
dtype: float64


## Poistetaan tyhjiä arvoja

Poistetaan rivit, joissa esiintyy vain vähän tyhjiä arvoja. Sarakkeet miltä poistetaan tyhjät rivit: 'body_type','exterior_color', 'seller_rating', 'transmission', 'fuel_type', 'engine_type', 'mileage', 'wheel_system'

In [5]:
df = df.dropna(subset=['body_type','exterior_color', 'seller_rating', 'transmission', 'fuel_type', 'engine_type', 'mileage', 'wheel_system'])
df.shape

(2588305, 23)

## Sarakkeiden tyhjien arvojen muuttaminen

**Fuel tank volume**

Poimitaan numeerisen osan alkuperäisistä arvoista, muunnetaan ne liukulukuiksi ja täytetään puuttuvat arvot vastaavan polttoainetyypin keskimääräisellä polttoainesäiliön tilavuudella.

In [6]:
# Fuel tank volume
df['fuel_tank_volume'] = df['fuel_tank_volume'].str.split(' ').str[0]
df['fuel_tank_volume'] = pd.to_numeric(df['fuel_tank_volume'], downcast='float', errors='coerce')
df['fuel_tank_volume'].fillna(df.groupby('fuel_type')['fuel_tank_volume'].transform('mean'), inplace=True)

**Exterior color**

Lasketaan jokaisen ulkovärin taajuuden (osuuden) sarakkeessa "exterior_color", korvataan todelliset väriarvot vastaavilla taajuuksilla.

In [7]:
# Exterior color
# Calculate the frequency of each color
color_frequency = df['exterior_color'].value_counts(normalize=True)  # normalize=True gives the proportion

# Map each color to its frequency
df['exterior_color'] = df['exterior_color'].map(color_frequency)

# Display the first few rows to verify the changes
print(df.head())

         body_type  city_fuel_economy engine_type  exterior_color  \
0  SUV / Crossover                NaN          I4        0.000098   
1  SUV / Crossover                NaN          I4        0.000449   
3  SUV / Crossover                NaN          V6        0.000038   
4  SUV / Crossover                NaN          I4        0.000449   
5  SUV / Crossover                NaN          I4        0.000012   

  frame_damaged  fuel_tank_volume fuel_type has_accidents  \
0           NaN         12.700000  Gasoline           NaN   
1           NaN         17.700001  Gasoline           NaN   
3           NaN         23.500000  Gasoline           NaN   
4           NaN         17.700001  Gasoline           NaN   
5         False         16.600000  Gasoline         False   

   highway_fuel_economy  horsepower  ... maximum_seating mileage  \
0                   NaN       177.0  ...         5 seats     7.0   
1                   NaN       246.0  ...         7 seats     8.0   
3             

**isCab**

Sarakkeen isCab puuttuvat arvot täytetään epätodella. Muutetaan 'frame_damaged', 'has_accidents', 'salvage' sarakkeet 'has_accidents' sarakkeeksi käyttäen apuna isCab saraketta, koska 'frame_damaged', 'has_accidents', 'salvage' ja 'isCab' sarakkeet sisältävät täysin samat tiedot ja tyhjien prosenttiosuuden.

In [8]:
df = df.drop(['frame_damaged', 'has_accidents', 'salvage'], axis=1)
df['isCab'].fillna(False, inplace=True)
df['damage_history'] = df['isCab']

**city_fuel_economy** ja **highway_fuel_economy**

Täytetään sarakkeiden puuttuvat arvot sarakkeen keskiarvolla.

In [9]:
# City fuel economy
df['city_fuel_economy'].replace(np.nan, df['city_fuel_economy'].mean(), inplace=True)

# highway fuel economy
df['highway_fuel_economy'].fillna(df['highway_fuel_economy'].mean(), inplace=True)

**horsepower**

Täytetään puuttuvat arvot sarakkeen modella.

In [10]:
# Horsepower
df['horsepower'].fillna(df['horsepower'].mode()[0], inplace=True)

**major options**

Luodaan uusi sarake ('major_options_count'), joka laskee tärkeimpien vaihtoehtojen määrän jokaiselle merkinnälle laskemalla pilkut 'major_options'-sarakkeessa, täyttämällä kaikki puuttuvat arvot nollalla ja poistamalla sitten alkuperäinen "major_options" -sarake DataFramesta. Tuloksena olevassa DataFrame-kehyksessä on päävaihtoehtojen numeerinen määrä jokaiselle merkinnälle "major_options_count" -sarakkeessa.

In [11]:
# Major options
df['major_options_count'] = df['major_options'].str.count(',') + 1
df['major_options_count'].fillna(0, inplace=True)
df.drop(['major_options'], axis=1, inplace=True)

**maximum seating**

Korvataan kaikki "--" esiintymät sanalla NaN ja täytetään tyhjät arvot 5 paikalla. Poimitaan numeerisen osan (ensimmäisen sanan) kustakin "maximum_seating" -sarakkeen merkinnästä.

In [12]:
# Maximum seating
df['maximum_seating'].replace("--", np.nan , inplace=True)
df['maximum_seating'].fillna("5 seats", inplace=True)
df['maximum_seating'] = df['maximum_seating'].str.split(' ').str[0]

**mileage**

Poistetaan outlier rivit sarakkeesta.

In [13]:
# Mileage
q1 = df['mileage'].quantile(0.25)
q3 = df['mileage'].quantile(0.75)
iqr = q3 - q1
df = df[(df['mileage'] >= q1 - 1.5*iqr) & (df['mileage'] <= q3 + 1.5*iqr)]

**torque**

Poimitaan numeerinen osa (ensimmäisen sanan) jokaisesta syötteestä, muunnetaan arvot liukuluvuiksi ja täytetään puuttuvat arvot keskimääräisellä vääntömomentilla.

In [14]:
# Torque
df['torque'] = df['torque'].str.split(" ").str[0]
df['torque'] = df['torque'].astype(float)
df['torque'].fillna(df['torque'].mean(), inplace=True)

In [15]:
print((df.isna().sum()/df.shape[0]*100).sort_values(ascending=False))

body_type               0.0
mileage                 0.0
damage_history          0.0
year                    0.0
wheel_system            0.0
transmission            0.0
torque                  0.0
seller_rating           0.0
price                   0.0
model_name              0.0
maximum_seating         0.0
city_fuel_economy       0.0
make_name               0.0
isCab                   0.0
horsepower              0.0
highway_fuel_economy    0.0
fuel_type               0.0
fuel_tank_volume        0.0
exterior_color          0.0
engine_type             0.0
major_options_count     0.0
dtype: float64


In [16]:
df.head()

,body_type,city_fuel_economy,engine_type,exterior_color,fuel_tank_volume,fuel_type,highway_fuel_economy,horsepower,isCab,make_name,...,mileage,model_name,price,seller_rating,torque,transmission,wheel_system,year,damage_history,major_options_count
0,SUV / Crossover,22.270333,I4,0.000098,12.700000,Gasoline,29.161924,177.0,False,Jeep,...,7.0,Renegade,23141.0,2.8,200.0,A,FWD,2019,False,1.0
1,SUV / Crossover,22.270333,I4,0.000449,17.700001,Gasoline,29.161924,246.0,False,Land Rover,...,8.0,Discovery Sport,46500.0,3.0,269.0,A,AWD,2020,False,1.0
3,SUV / Crossover,22.270333,V6,0.000038,23.500000,Gasoline,29.161924,340.0,False,Land Rover,...,11.0,Discovery,67430.0,3.0,332.0,A,AWD,2020,False,0.0
4,SUV / Crossover,22.270333,I4,0.000449,17.700001,Gasoline,29.161924,246.0,False,Land Rover,...,7.0,Discovery Sport,48880.0,3.0,269.0,A,AWD,2020,False,1.0
5,SUV / Crossover,22.270333,I4,0.000012,16.600000,Gasoline,29.161924,247.0,False,Land Rover,...,12.0,Range Rover Velar,66903.0,3.0,269.0,A,AWD,2020,False,8.0


**model_name**

Lasketaan jokaisen mallin taajuus (osuus) sarakkeessa "mosel_name", korvataan todelliset mallit vastaavilla taajuuksilla.

In [17]:
# Calculate the frequency of each model name
model_frequency = df['model_name'].value_counts(normalize=True)  # normalize=True gives the proportion

# Map each model name to its frequency
df['model_name'] = df['model_name'].map(model_frequency)

df.head()

,body_type,city_fuel_economy,engine_type,exterior_color,fuel_tank_volume,fuel_type,highway_fuel_economy,horsepower,isCab,make_name,...,mileage,model_name,price,seller_rating,torque,transmission,wheel_system,year,damage_history,major_options_count
0,SUV / Crossover,22.270333,I4,0.000098,12.700000,Gasoline,29.161924,177.0,False,Jeep,...,7.0,0.006273,23141.0,2.8,200.0,A,FWD,2019,False,1.0
1,SUV / Crossover,22.270333,I4,0.000449,17.700001,Gasoline,29.161924,246.0,False,Land Rover,...,8.0,0.001287,46500.0,3.0,269.0,A,AWD,2020,False,1.0
3,SUV / Crossover,22.270333,V6,0.000038,23.500000,Gasoline,29.161924,340.0,False,Land Rover,...,11.0,0.000475,67430.0,3.0,332.0,A,AWD,2020,False,0.0
4,SUV / Crossover,22.270333,I4,0.000449,17.700001,Gasoline,29.161924,246.0,False,Land Rover,...,7.0,0.001287,48880.0,3.0,269.0,A,AWD,2020,False,1.0
5,SUV / Crossover,22.270333,I4,0.000012,16.600000,Gasoline,29.161924,247.0,False,Land Rover,...,12.0,0.000567,66903.0,3.0,269.0,A,AWD,2020,False,8.0


**price**

Poistetaan suurimmat esiintymät hinta -sarakkeesta.

In [18]:
Q1 = df['price'].quantile(0.01)
Q3 = df['price'].quantile(0.99)
IQR = Q3 - Q1

# Define the upper bound for outliers
upper_bound = Q3 + 1.5 * IQR

# Filter out rows with 'price' values above the upper bound
df = df[df['price'] <= upper_bound]

**engine_type**

Poimitaan vain moottorin tyyppi merkkijonosta.

In [19]:
df['engine_type'] = df['engine_type'].str.split(' ').str[0]
df['engine_type'].unique()

array(['I4', 'V6', 'I6', 'V8', 'H4', 'I3', 'I5', 'R2', 'W12', 'V10',
       'V12', 'H6', 'I2', 'W8'], dtype=object)

## Muutetaan kategoriset arvot one hot encodingia käyttäen

In [20]:
df_dummy = pd.get_dummies(df)

## Tallennetaan dataframe csv -tiedostona

In [21]:
df.shape

(2384759, 21)

In [22]:
df_dummy.shape

(2384759, 125)

In [23]:
df.to_csv('./temp/cleaned_data.csv', index=False)
df_dummy.to_csv('./temp/cleaned_data2.csv', index=False)